In [0]:
use catalog uc_prod

### Getting the distinct states from the staing master table (silver.daily_pricing_silver)

In [0]:
insert into gold.reporting_fact_daily_pricing_gold
select 
dt.DATE_ID,
st.state_id,
mkt.MARKET_ID,
prd.PRODUCT_ID,
vr.VARIETY_ID,
dps.ROW_ID,
dps.ARRIVAL_IN_TONNES,
dps.minimum_price,
dps.maximum_price,
dps.modal_price,
current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date
from silver.daily_pricing_silver as dps
left join gold.reporting_dim_date_gold as dt on dps.date_of_pricing = dt.CALENDAR_DATE
left join gold.reporting_dim_market_gold as mkt on dps.market_name = mkt.MARKET_NAME
left join gold.reporting_dim_state_gold as st on dps.STATE_NAME = st.STATE_NAME
left join gold.reporting_dim_variety_gold as vr on dps.variety = vr.variety
left join gold.reporting_dim_product_gold as prd on dps.product_name  = prd.PRODUCT_NAME and dps.productgroup_name = prd.PRODUCTGROUP_NAME
where record_updated_date > (
      select nvl(max(Process_file_updated_date),'2010-01-01') from uc_prod.watermark_tbl.process_logs 
      where process_name = 'reporting_fact_daily_pricing_gold' and process_status ='completed'
                            )
 

### Add the distinct unique id for the state items

### update the water mark table

In [0]:
insert into uc_prod.watermark_tbl.process_logs
(Process_name,Process_file_updated_date,process_status)
values
(
'reporting_fact_daily_pricing_gold',
current_timestamp(),
'completed'
)